# Тема 3. Дифференцирование таблично заданной функции с помощью многочлена Лагранжа

In [159]:
# %run LR1.ipynb

In [160]:
import numpy as np
import math
import pandas as pd
import sympy as sp

Подготовка

In [161]:
# Символьные переменные
x = sp.Symbol('x')
xi = sp.Symbol('xi')  # для остаточного члена
h = sp.Symbol('h', positive=True)


In [162]:
f_sym = (x-1)**2 - 0.5*sp.exp(x)
f_prime_sym = sp.diff(f_sym, x)
f_double_prime_sym = sp.diff(f_prime_sym, x)

In [163]:
print("Символьное представление функции и производных:")
print(f"f(x) = {f_sym}")
print(f"f'(x) = {f_prime_sym}")
print(f"f''(x) = {f_double_prime_sym}")


Символьное представление функции и производных:
f(x) = (x - 1)**2 - 0.5*exp(x)
f'(x) = 2*x - 0.5*exp(x) - 2
f''(x) = 2 - 0.5*exp(x)


In [164]:
a, b = 0.1, 0.6
n_total = 11
h_num = (b - a) / (n_total - 1)
x_nodes = [a + i*h_num for i in range(n_total)]




In [165]:
# Параметры задачи
k = 2  # порядок производной
n = 5  # степень многочлена
m = 1  # индекс точки

In [166]:
# Выбираем узлы вокруг точки x₁
n_temp = n + 1

start_idx = end_idx = m
while (end_idx - start_idx+1) < n_temp:
    start_idx = max(0, start_idx-1)
    end_idx = min(len(x_nodes)-1, end_idx+1)

if (end_idx - start_idx+1) > n_temp:
    if (m-start_idx) >= (end_idx - m):
        start_idx += 1
    else:
        end_idx -= 1

selected_nodes = x_nodes[start_idx:end_idx + 1]
selected_f_vals = [f_sym.subs({"x": x}) for x in selected_nodes]

Построение многочлена Лагранжа

In [167]:
def lagrange_basis(i):
    """Символьный базисный полином Лагранжа"""
    result = 1
    for j in range(len(selected_nodes)):
        if j != i:
            result *= (x - selected_nodes[j]) / ((i - j) * h_num)
    return result

def lagrange_polynomial():
    """Символьный интерполяционный многочлен Лагранжа"""
    L_n = 0
    for i in range(len(selected_nodes)):
        basis = lagrange_basis(i)
        L_n += selected_f_vals[i] * basis
    return sp.simplify(L_n)


print(f"Выбранные узлы (n={n}):")
for i, node in enumerate(selected_nodes):
    print(f"x_{start_idx + i} = {node:.3f}, f = {selected_f_vals[i]:.6f}")

# Строим символьный многочлен Лагранжа
L_n_sym = lagrange_polynomial()
print(f"Многочлен Лагранжа L_{n}(x):")
print(L_n_sym)

Выбранные узлы (n=5):
x_0 = 0.100, f = 0.257415
x_1 = 0.150, f = 0.141583
x_2 = 0.200, f = 0.029299
x_3 = 0.250, f = -0.079513
x_4 = 0.300, f = -0.184929
x_5 = 0.350, f = -0.287034
Многочлен Лагранжа L_5(x):
-0.00522072969124565*x**5 - 0.0202357655425658*x**4 - 0.0835050303808202*x**3 + 0.750026592004588*x**2 - 2.500002105152*x + 0.500000066371568


Аналитическое дифференцирование многочлена Лагранжа

In [168]:
# Вычисляем k-ю производную аналитически
L_n_deriv_sym = sp.diff(L_n_sym, x, k)
L_n_deriv_sym = sp.simplify(L_n_deriv_sym)

print(f"Аналитическая {k}-я производная многочлена Лагранжа:")
print(f"L_{n}^({k})(x) = {L_n_deriv_sym}")

# Вычисляем значение производной в точке x_m
x_m = x_nodes[m]
L_deriv_value = L_n_deriv_sym.subs(x, x_m)
L_deriv_value = float(L_deriv_value)  # преобразуем в число

# Точное значение производной
f_exact_deriv_value = float(f_double_prime_sym.subs(x, x_m))

print(f"РЕЗУЛЬТАТЫ ДИФФЕРЕНЦИРОВАНИЯ:")
print(f"Точка x_{m} = {x_m:.6f}")
print(f"L_{n}^({k})(x_{m}) = {L_deriv_value:.10f}")
print(f"f^({k})(x_{m}) = {f_exact_deriv_value:.10f}")
print(f"Погрешность R_{n},{k} = {L_deriv_value - f_exact_deriv_value:.10f}")

Аналитическая 2-я производная многочлена Лагранжа:
L_5^(2)(x) = -0.104414593824913*x**3 - 0.242829186510789*x**2 - 0.501030182284921*x + 1.50005318400918
РЕЗУЛЬТАТЫ ДИФФЕРЕНЦИРОВАНИЯ:
Точка x_1 = 0.150000
L_5^(2)(x_1) = 1.4190826007
f^(2)(x_1) = 1.4190828786
Погрешность R_5,2 = -0.0000002779


Аналитическое вычисление остаточного члена

In [169]:
def symbolic_omega(nodes):
    """Символьная функция ω_{n+1}(x) = (x - x₀)(x - x₁)...(x - xₙ)"""
    result = 1
    for node in nodes:
        result *= (x - node)
    return sp.simplify(result)

def symbolic_derivative_error(nodes, degree, deriv_order):
    """Аналитическое вычисление остаточного члена для производной"""
    n = degree
    
    # ω_{n+1}(x)
    omega_n1 = symbolic_omega(nodes)
    
    # k-я производная от ω_{n+1}(x)
    omega_deriv = sp.diff(omega_n1, x, deriv_order)
    omega_deriv = sp.simplify(omega_deriv)
    
    # f^{(n+1)}(ξ) - (n+1)-я производная исходной функции
    f_deriv_n1 = sp.diff(f_sym, x, n + 1)
    
    # Остаточный член: R_{n,k}(x) = f^{(n+1)}(ξ)/(n+1)! · ω_{n+1}^{(k)}(x)
    R_nk = (f_deriv_n1 / sp.factorial(n + 1)) * omega_deriv
    
    return sp.simplify(R_nk), omega_deriv, f_deriv_n1

# Вычисляем остаточный член аналитически
R_nk_sym, omega_deriv_sym, f_deriv_n1_sym = symbolic_derivative_error(selected_nodes, n, k)

print(f"АНАЛИТИЧЕСКИЙ ОСТАТОЧНЫЙ ЧЛЕН:")
print(f"ω_{n+1}^({k})(x) = {omega_deriv_sym}")
print(f"f^({n+1})(ξ) = {f_deriv_n1_sym}")
print(f"R_{n},{k}(x) = {R_nk_sym}")


АНАЛИТИЧЕСКИЙ ОСТАТОЧНЫЙ ЧЛЕН:
ω_6^(2)(x) = 30.0*x**4 - 27.0*x**3 + 8.85*x**2 - 1.24875*x + 0.0638
f^(6)(ξ) = -0.5*exp(x)
R_5,2(x) = (-0.0208333333333333*x**4 + 0.01875*x**3 - 0.00614583333333334*x**2 + 0.0008671875*x - 4.43055555555556e-5)*exp(x)


Оценка границ остаточного члена

In [170]:
def evaluate_error_bounds(R_nk_sym, nodes, x_point):
    """Оценка минимального и максимального значения остаточного члена"""
    
    # f^{(n+1)}(ξ) на отрезке [min(nodes), max(nodes)]
    f_deriv_n1_num = sp.lambdify(x, f_deriv_n1_sym, 'numpy')
    deriv_values = [f_deriv_n1_num(node) for node in [min(nodes), max(nodes)]]
    f_deriv_min = min(deriv_values)
    f_deriv_max = max(deriv_values)
    
    # ω_{n+1}^{(k)}(x) в точке x_point
    omega_deriv_num = sp.lambdify(x, omega_deriv_sym, 'numpy')
    omega_deriv_value = omega_deriv_num(x_point)
    
    # Границы остаточного члена
    R_min = (f_deriv_min * omega_deriv_value) / math.factorial(n + 1)
    R_max = (f_deriv_max * omega_deriv_value) / math.factorial(n + 1)
    R_min, R_max = min(R_min, R_max), max(R_min, R_max)
    return R_min, R_max

# Оцениваем границы погрешности
R_min, R_max = evaluate_error_bounds(R_nk_sym, selected_nodes, x_m)
R_real = L_deriv_value - f_exact_deriv_value

print(f"ОЦЕНКА ГРАНИЦ ПОГРЕШНОСТИ:")
print(f"min R_{n},{k} = {R_min:.10f}")
print(f"max R_{n},{k} = {R_max:.10f}")
print(f"реальный остаточный член = {R_real:.10f}")
print(f"Фактическая погрешность = {R_real:.10e}")
print(f"min R_{n},{k} < R_{n},{k} < max R_{n},{k}: {(R_min) < abs(R_real) < (R_max)}")


ОЦЕНКА ГРАНИЦ ПОГРЕШНОСТИ:
min R_5,2 = 0.0000002494
max R_5,2 = 0.0000003203
реальный остаточный член = -0.0000002779
Фактическая погрешность = -2.7792007318e-07
min R_5,2 < R_5,2 < max R_5,2: True
